# 0 - Introduction

Import libraries

In [12]:
import numpy
import scipy.linalg
import numpy.linalg
import scipy.special
import matplotlib.pyplot
import seaborn
import scipy.optimize

Utility functions

In [2]:
# ----- Convert an array into a column -----
def row_to_column (array):
    return array.reshape((array.size, 1))


# ----- Convert an array into a row -----
def column_to_row (array):
    return array.reshape((1, array.size))


# ----- Compute mean -----
def compute_mean (data):
    return data.mean(1)


# ----- Compute covariance -----
def compute_covariance (data):
    mu = compute_mean(data)
    dc = data - mu.reshape((mu.size, 1))
    cov = numpy.dot(dc, dc.T) / float(data.shape[1])
    return cov


# ----- Compute Pearson correlation -----
def compute_correlation(x, y):
    x1_sum = numpy.sum(x)
    y1_sum = numpy.sum(y)
    x2_sum = numpy.sum(x**2)
    y2_sum = numpy.sum(y**2)
    cp_sum = numpy.sum(x * y.T)
    n = x.shape[0]

    num = n*cp_sum - x1_sum*y1_sum
    den = numpy.sqrt((n*x2_sum - x1_sum**2) * (n*y2_sum - y1_sum**2))
    return (num/den)


# ----- Compute logarithm of probability density function of a Gaussian distribution -----
def logpdf_GAU_ND (X, mu, C):
    _,det = numpy.linalg.slogdet(C)
    inv = numpy.linalg.inv(C)

    term1 = -0.5 * X.shape[0] * numpy.log(2*numpy.pi)
    term2 = -0.5 * det
    term3 = -0.5 * numpy.dot((X-mu).T, numpy.dot(inv, (X-mu))).sum(0)

    return (term1+term2+term3)


# ----- Classify samples matching log-likelihood ratio and a threshold -----
def predict_labels (llr, threshold):
    predicted = numpy.zeros(len(llr))
    for i in range(len(llr)):
        if (llr[i]>threshold): predicted[i] = 1
    return predicted


# ----- Count the number of mispredictions -----
def count_mispredictions (predicted, LTE):
    wrong = 0
    for i in range(len(LTE)):
        if (predicted[i]!=LTE[i]): wrong += 1
    return wrong


# ----- Square a matrix and transpose it -----
def square_and_transpose (matrix):
    x = matrix[:, None]
    xxT = x.dot(x.T).reshape((x.size)**2, order='F')
    return xxT

# 1 - Dataset

## 1.1 - Load dataset

In [13]:
# ----- Global variables -----
features = int(10)
distinct_classes = int(2)
training_input = "../data/Train.txt"
test_input = "../data/Test.txt"


# ----- Read file -----
def read_file (filename):
    D = []
    L = []
    with open(filename) as file:
        for line in file:
            try:
                attributes = line.split(",")[0:features]
                attributes = row_to_column(numpy.array([float(i) for i in attributes]))
                label = int(line.split(",")[-1].strip())
                D.append(attributes)
                L.append(label)
            except:
                pass
    return numpy.hstack(D), numpy.array(L, dtype=numpy.int32)


# ----- Load training set -----
def load_training_set ():
    DTR,LTR = read_file(training_input)
    return DTR,LTR


# ----- Load test set -----
def load_test_set ():
    DTE,LTE = read_file(test_input)
    return DTE,LTE


# ----- Instructions -----
DTR,LTR = load_training_set(training_input)
DTE,LTE = load_test_set(test_input)

## 1.2 - Features analysis

In [ ]:
# ----- Global variables -----
histograms_folder = "../output/00_Histograms"
heatmaps_folder = "../output/01_Heatmaps"


# ----- Plot histograms of training set -----
def plot_dataset_histograms (DTR, LTR):
    # Split according to label
    D0 = DTR[:, LTR==0]
    D1 = DTR[:, LTR==1]

    # Plot histograms for each feature
    for x in range(features):
        matplotlib.pyplot.figure()
        matplotlib.pyplot.title("Feature %d" % x)
        matplotlib.pyplot.hist(D0[x,:], bins=40, density=True, alpha=0.4, edgecolor="black", label="Spoofed")
        matplotlib.pyplot.hist(D1[x,:], bins=40, density=True, alpha=0.4, edgecolor="black", label="Authentic")
        matplotlib.pyplot.legend()
        matplotlib.pyplot.savefig("%s/histogram_%d.png" % (histograms_folder, x))
        matplotlib.pyplot.close()


# ----- Plot heatmaps of training set ---
def plot_dataset_heatmaps (DTR, LTR):
    # Consider all samples
    corr = numpy.zeros((features, features))
    for x in range(features):
        for y in range(features):
            corr[x][y] = compute_correlation(DTR[x,:], DTR[y,:])
    seaborn.set()
    heatmap = seaborn.heatmap(numpy.abs(corr), cmap="YlGnBu", linewidth=0.3, square=True, cbar=False)
    figure = heatmap.get_figure()
    figure.savefig("%s/heatmap_all.png" % (heatmaps_folder))

    # Consider only samples labeled as spoofed fingerprints
    corr = numpy.zeros((features, features))
    for x in range(features):
        for y in range(features):
            corr[x][y] = compute_correlation(DTR[x,LTR==0], DTR[y,LTR==0])
    seaborn.set()
    heatmap = seaborn.heatmap(numpy.abs(corr), cmap="coolwarm", linewidth=0.3, square=True, cbar=False)
    figure = heatmap.get_figure()
    figure.savefig("%s/heatmap_spoofed.png" % (heatmaps_folder))

    # Consider only samples labeled as authentic fingerprints
    corr = numpy.zeros((features, features))
    for x in range(features):
        for y in range(features):
            corr[x][y] = compute_correlation(DTR[x,LTR==1], DTR[y,LTR==1])
    seaborn.set()
    heatmap = seaborn.heatmap(numpy.abs(corr), cmap="BuPu", linewidth=0.3, square=True, cbar=False)
    figure = heatmap.get_figure()
    figure.savefig("%s/heatmap_authentic.png" % (heatmaps_folder))


# ----- Instructions -----
plot_dataset_histograms(DTR, LTR)
plot_dataset_heatmaps(DTR, LTR)

## 1.3 - Dimensionality reduction

In [ ]:
# ----- Global variables -----
pca_folder = "output/02_PCA"
lda_folder = "output/03_LDA"


# ----- Principal Component Analysis -----
def apply_pca (D, L, m, output_folder=None):
    # Identify the largest eigenvalues and eigenvectors over to which we project original data
    _,U = numpy.linalg.eigh(utility.compute_covariance(D))
    P = U[:, ::-1][:, 0:m]
    
    if output_folder!=None:
        # Project data
        DP = numpy.dot(P.T, D)
        plot_pca_scatters(DP, L, output_folder)

    return P


# ----- Linear Discriminant Analysis -----
def apply_lda (D, L, m, output_folder=None):
    # Compute between- and within-class covariance matrices
    D0 = D[:, L==0]
    D1 = D[:, L==1]
    diff0 = compute_mean(D0) - compute_mean(D)
    diff1 = compute_mean(D1) - compute_mean(D)
    SB = numpy.outer(diff0,diff0) + numpy.outer(diff1,diff1)
    SW = compute_covariance(D0) + compute_covariance(D1)
    
    # Identify the m largest eigenvalues and eigenvectors over to which we project original data
    U,s,_ = numpy.linalg.svd(SW)
    P1 = numpy.dot(U, row_to_column(1.0/s**0.5)*U.T)
    SBt = numpy.dot(P1, numpy.dot(SB, P1.T))
    U,_,_ = numpy.linalg.svd(SBt)
    P2 = U[:, 0:m]
    W = numpy.dot(P1.T, P2)

    if output_folder!=None:
        # Project data
        DP = numpy.dot(W.T, D)
        plot_lda_histograms(DP, L, output_folder)


# ----- Plot scatters of data projected after PCA -----
def plot_pca_scatters (D, L, output_folder):
    D0 = D[:, L==0]
    D1 = D[:, L==1]

    matplotlib.pyplot.figure()
    matplotlib.pyplot.scatter(D0[0,:], D0[1,:], label="Spoofed")
    matplotlib.pyplot.scatter(D1[0,:], D1[1,:], label="Authentic")
    matplotlib.pyplot.legend()
    matplotlib.pyplot.savefig("%s/scatter_pca.png" % (output_folder))
    matplotlib.pyplot.close()


# ----- Plot histograms of data projected after LDA -----
def plot_lda_histograms (D, L, output_folder):
    # Split according to label
    D0 = D[:, L==0]
    D1 = D[:, L==1]

    # Plot histograms for each feature
    matplotlib.pyplot.figure()
    matplotlib.pyplot.hist(D0[0,:], bins=40, density=True, alpha=0.4, edgecolor="black", label="Spoofed")
    matplotlib.pyplot.hist(D1[0,:], bins=40, density=True, alpha=0.4, edgecolor="black", label="Authentic")
    matplotlib.pyplot.legend()
    matplotlib.pyplot.savefig("%s/histogram_lda.png" % (output_folder))
    matplotlib.pyplot.close()


# ----- Plot scatters of data projected after LDA -----
def plot_lda_scatters (D, L, output_folder):
    D0 = D[:, L==0]
    D1 = D[:, L==1]

    matplotlib.pyplot.figure()
    matplotlib.pyplot.scatter(D0[0,:], D0[1,:], label="Spoofed")
    matplotlib.pyplot.scatter(D1[0,:], D1[1,:], label="Authentic")
    matplotlib.pyplot.legend()
    matplotlib.pyplot.savefig("%s/scatter_lda.png" % (output_folder))
    matplotlib.pyplot.close()
    

# ----- Instructions -----
m = 2
apply_pca(DTR, LTR, m, pca_folder)
apply_lda(DTR, LTR, m, lda_folder)

# 2 - Training

## 2.1 - Generative models

### 2.1.0 - Intro

Computation of costs

In [29]:
# ----- Compute un-normalized Detection Cost Function -----
def compute_unnormalized_DCF (pi_t, Cfn, Cfp, ll_ratios, LTE, threshold=None, is_effective=False):
    # Define threshold if not previously defined
    if threshold is None:
        if is_effective is False:
            effective_prior = (pi_t*Cfn) / (pi_t*Cfn + (1-pi_t)*Cfp)
        else:
            effective_prior = pi_t
        threshold = -numpy.log(effective_prior / (1-effective_prior))
    
    # Build confusion matrix
    predicted = (ll_ratios>threshold).astype(int)
    confusion_matrix = numpy.zeros((2,2))
    for i in range(len(LTE)):
        confusion_matrix[predicted[i], LTE[i].astype(int)] += 1
    
    # Compute DCF
    fnr = confusion_matrix[0][1] / (confusion_matrix[0][1]+confusion_matrix[1][1])
    fpr = confusion_matrix[1][0] / (confusion_matrix[0][0]+confusion_matrix[1][0])
    dcf = pi_t*Cfn*fnr + (1-pi_t)*Cfp*fpr

    return dcf


# ----- Compute normalized Detection Cost Function -----
def compute_normalized_DCF (pi_t, Cfn, Cfp, DCFu):
    dummy_costs = numpy.array([pi_t*Cfn, (1-pi_t)*Cfp])
    index = numpy.argmin(dummy_costs)
    dcf = DCFu / dummy_costs[index]
    return dcf


# ----- Compute actual Detection Cost Function -----
def compute_actual_DCF (pi_t, Cfn, Cfp, ll_ratios, LTE, is_effective):
    dcfu = compute_unnormalized_DCF(pi_t, Cfn, Cfp, ll_ratios, LTE, None, is_effective)
    dcf = compute_normalized_DCF(pi_t, Cfn, Cfp, dcfu)
    return dcf


# ----- Compute minimum Detection Cost Function -----
def compute_min_DCF (pi_t, Cfn, Cfp, ll_ratios, LTE):
    dcf_collection = numpy.zeros(ll_ratios.shape)
    sorted = numpy.sort(ll_ratios)

    for i in range(len(ll_ratios)):
        threshold = sorted[i]
        unnormalized = compute_unnormalized_DCF(pi_t, Cfn, Cfp, ll_ratios, LTE, threshold)
        dcf_collection[i] = compute_normalized_DCF(pi_t, Cfn, Cfp, unnormalized)
    return numpy.min(dcf_collection)

K-Fold implementation

In [35]:
# ----- Global variables -----
K = 5
Cfn = 1
Cfp = 10
generative_filename = "output/04_Training/Generative.txt"


# ----- Compare classifiers using K-Fold -----
def gen_kfold (D, L, pca_value, pi_value, function, name):
    N = int(D.shape[1]/K)
    wrong_predictions = 0
    numpy.random.seed(2)
    ll_ratios = []
    labels = []
    indexes = numpy.random.permutation(D.shape[1])

    for i in range(K):
        # Select which subset to use for evaluation
        idxTest = indexes[i*N:(i+1)*N]
        if i>0: idxTrainLeft = indexes[0:i*N]
        elif (i+1)<K: idxTrainRight = indexes[(i+1)*N:]
        if i==0: idxTrain = idxTrainRight
        elif (i+1)==K: idxTrain = idxTrainLeft
        else: idxTrain = numpy.hstack([idxTrainLeft, idxTrainRight])
        DTR,LTR = D[:,idxTrain], L[idxTrain]
        DTE,LTE = D[:,idxTest], L[idxTest]

        # Apply PCA if necessary
        if pca_value!=0:
            P = apply_pca(DTR, LTR, pca_value)
            DTR,DTE = numpy.dot(P.T, DTR), numpy.dot(P.T, DTE)

        # Apply classifier
        wrong, llr = function(DTR, LTR, DTE, LTE)
        wrong_predictions += wrong
        ll_ratios.append(llr)
        labels.append(LTE)

    # Evaluate accuracy, error rate, and minDCF
    error_rate = wrong_predictions / D.shape[1]
    accuracy = 1 - error_rate
    cost = compute_min_DCF(pi_value, Cfn, Cfp, numpy.hstack(ll_ratios), numpy.hstack(labels))

    # Print results
    print("%s" % (name))
    if (pca_value==0): print("  No PCA, pi = %.2f" % (pi_value))
    else: print("  PCA = %d, pi = %.2f" % (pca_value, pi_value))
    print("  Accuracy: %.3f%%" % (100.0*accuracy))
    print("  Error rate: %.3f%%" % (100.0*error_rate))
    print("  min DCF: %.3f" % (cost))
    print("\n")

    # Save results to file
    with open(generative_filename, "a") as file:
        file.write("%s\n" % (name))
        if (pca_value==0): file.write("  No PCA, pi = %.2f\n" % (pi_value))
        else: file.write("  PCA = %d, pi = %.2f\n" % (pca_value, pi_value))
        file.write("  Accuracy: %.3f%%\n" % (100.0*accuracy))
        file.write("  Error rate: %.3f%%\n" % (100.0*error_rate))
        file.write("  min DCF: %.3f\n" % (cost))
        file.write("\n\n")

### 2.1.1 - Multivariate Gaussian without tied covariance

In [42]:
# ----- Multivariate Gaussian without tied covariance -----
def mvg (DTR, LTR, DTE, LTE):
    # Compute mean and covariance for each class
    D0 = DTR[:, LTR==0]
    D1 = DTR[:, LTR==1]
    mu0, cov0 = row_to_column(compute_mean(D0)), compute_covariance(D0)
    mu1, cov1 = row_to_column(compute_mean(D1)), compute_covariance(D1)

    # Compute likelihoods
    S = numpy.empty((2,DTE.shape[1]))
    for i in range(DTE.shape[1]):
        S[0,i] = numpy.exp(logpdf_GAU_ND(row_to_column(DTE[:,i]), mu0, cov0))
        S[1,i] = numpy.exp(logpdf_GAU_ND(row_to_column(DTE[:,i]), mu1, cov1))
    
    # Compute log-likelihood ratio and use it to classify samples
    llr = numpy.log(S[1,:] / S[0,:])
    predicted_labels = numpy.log(S).argmax(axis=0)
    wrong_predictions = len(LTE) - numpy.array(predicted_labels == LTE).sum()

    return wrong_predictions, llr

No PCA, &pi; = 0.1

In [ ]:
pca_value = 0
pi_value = 0.1
gen_kfold(DTR, LTR, pca_value, pi_value, mvg, "Multivariate Gaussian classifier without tied covariance")

No PCA, &pi; = 0.5

In [ ]:
pca_value = 0
pi_value = 0.5
gen_kfold(DTR, LTR, pca_value, pi_value, mvg, "Multivariate Gaussian classifier without tied covariance")

No PCA, &pi; = 0.9

In [ ]:
pca_value = 0
pi_value = 0.9
gen_kfold(DTR, LTR, pca_value, pi_value, mvg, "Multivariate Gaussian classifier without tied covariance")

PCA = 9, &pi; = 0.1

In [ ]:
pca_value = 9
pi_value = 0.1
gen_kfold(DTR, LTR, pca_value, pi_value, mvg, "Multivariate Gaussian classifier without tied covariance")

PCA = 9, &pi; = 0.5

In [ ]:
pca_value = 9
pi_value = 0.5
gen_kfold(DTR, LTR, pca_value, pi_value, mvg, "Multivariate Gaussian classifier without tied covariance")

PCA = 9, &pi; = 0.9

In [ ]:
pca_value = 9
pi_value = 0.9
gen_kfold(DTR, LTR, pca_value, pi_value, mvg, "Multivariate Gaussian classifier without tied covariance")

PCA = 8, &pi; = 0.1

In [ ]:
pca_value = 8
pi_value = 0.1
gen_kfold(DTR, LTR, pca_value, pi_value, mvg, "Multivariate Gaussian classifier without tied covariance")

PCA = 8, &pi; = 0.5

In [ ]:
pca_value = 8
pi_value = 0.5
gen_kfold(DTR, LTR, pca_value, pi_value, mvg, "Multivariate Gaussian classifier without tied covariance")

PCA = 8, &pi; = 0.9

In [ ]:
pca_value = 8
pi_value = 0.9
gen_kfold(DTR, LTR, pca_value, pi_value, mvg, "Multivariate Gaussian classifier without tied covariance")

PCA = 7, &pi; = 0.1

In [ ]:
pca_value = 7
pi_value = 0.1
gen_kfold(DTR, LTR, pca_value, pi_value, mvg, "Multivariate Gaussian classifier without tied covariance")

PCA = 7, &pi; = 0.5

In [ ]:
pca_value = 7
pi_value = 0.5
gen_kfold(DTR, LTR, pca_value, pi_value, mvg, "Multivariate Gaussian classifier without tied covariance")

PCA = 7, &pi; = 0.9

In [ ]:
pca_value = 7
pi_value = 0.9
gen_kfold(DTR, LTR, pca_value, pi_value, mvg, "Multivariate Gaussian classifier without tied covariance")

PCA = 6, &pi; = 0.1

In [ ]:
pca_value = 6
pi_value = 0.1
gen_kfold(DTR, LTR, pca_value, pi_value, mvg, "Multivariate Gaussian classifier without tied covariance")

PCA = 6, &pi; = 0.5

In [ ]:
pca_value = 6
pi_value = 0.5
gen_kfold(DTR, LTR, pca_value, pi_value, mvg, "Multivariate Gaussian classifier without tied covariance")

PCA = 6, &pi; = 0.9

In [ ]:
pca_value = 6
pi_value = 0.9
gen_kfold(DTR, LTR, pca_value, pi_value, mvg, "Multivariate Gaussian classifier without tied covariance")

PCA = 5, &pi; = 0.1

In [ ]:
pca_value = 5
pi_value = 0.1
gen_kfold(DTR, LTR, pca_value, pi_value, mvg, "Multivariate Gaussian classifier without tied covariance")

PCA = 5, &pi; = 0.5

In [ ]:
pca_value = 5
pi_value = 0.5
gen_kfold(DTR, LTR, pca_value, pi_value, mvg, "Multivariate Gaussian classifier without tied covariance")

PCA = 5, &pi; = 0.9

In [ ]:
pca_value = 5
pi_value = 0.9
gen_kfold(DTR, LTR, pca_value, pi_value, mvg, "Multivariate Gaussian classifier without tied covariance")

### 2.1.2 - Multivariate Gaussian with tied covariance

In [ ]:
# ----- Multivariate Gaussian with tied covariance -----
def tied_mvg (DTR, LTR, DTE, LTE):
    # Compute mean for each class and tied covariance matrix
    D0 = DTR[:, LTR==0]
    D1 = DTR[:, LTR==1]
    mu0, cov0 = row_to_column(compute_mean(D0)), compute_covariance(D0)
    mu1, cov1 = row_to_column(compute_mean(D1)), compute_covariance(D1)
    tied_cov = (cov0*D0.shape[1] + cov1*D1.shape[1]) / DTR.shape[1]

    # Compute likelihoods
    S = numpy.empty((2,DTE.shape[1]))
    for i in range(DTE.shape[1]):
        S[0,i] = numpy.exp(logpdf_GAU_ND(row_to_column(DTE[:,i]), mu0, tied_cov))
        S[1,i] = numpy.exp(logpdf_GAU_ND(row_to_column(DTE[:,i]), mu1, tied_cov))
    
    # Compute log-likelihood ratio and use it to classify samples
    llr = numpy.log(S[1,:] / S[0,:])
    predicted_labels = numpy.log(S).argmax(axis=0)
    wrong_predictions = len(LTE) - numpy.array(predicted_labels == LTE).sum()

    return wrong_predictions, llr

No PCA, &pi; = 0.1

In [ ]:
pca_value = 0
pi_value = 0.1
gen_kfold(DTR, LTR, pca_value, pi_value, tied_mvg, "Multivariate Gaussian classifier with tied covariance")

No PCA, &pi; = 0.5

In [ ]:
pca_value = 0
pi_value = 0.5
gen_kfold(DTR, LTR, pca_value, pi_value, tied_mvg, "Multivariate Gaussian classifier with tied covariance")

No PCA, &pi; = 0.9

In [ ]:
pca_value = 0
pi_value = 0.9
gen_kfold(DTR, LTR, pca_value, pi_value, tied_mvg, "Multivariate Gaussian classifier with tied covariance")

PCA = 9, &pi; = 0.1

In [ ]:
pca_value = 9
pi_value = 0.1
gen_kfold(DTR, LTR, pca_value, pi_value, tied_mvg, "Multivariate Gaussian classifier with tied covariance")

PCA = 9, &pi; = 0.5

In [ ]:
pca_value = 9
pi_value = 0.5
gen_kfold(DTR, LTR, pca_value, pi_value, tied_mvg, "Multivariate Gaussian classifier with tied covariance")

PCA = 9, &pi; = 0.9

In [ ]:
pca_value = 9
pi_value = 0.9
gen_kfold(DTR, LTR, pca_value, pi_value, tied_mvg, "Multivariate Gaussian classifier with tied covariance")

PCA = 8, &pi; = 0.1

In [ ]:
pca_value = 8
pi_value = 0.1
gen_kfold(DTR, LTR, pca_value, pi_value, tied_mvg, "Multivariate Gaussian classifier with tied covariance")

PCA = 8, &pi; = 0.5

In [ ]:
pca_value = 8
pi_value = 0.5
gen_kfold(DTR, LTR, pca_value, pi_value, tied_mvg, "Multivariate Gaussian classifier with tied covariance")

PCA = 8, &pi; = 0.9

In [ ]:
pca_value = 8
pi_value = 0.9
gen_kfold(DTR, LTR, pca_value, pi_value, tied_mvg, "Multivariate Gaussian classifier with tied covariance")

PCA = 7, &pi; = 0.1

In [ ]:
pca_value = 7
pi_value = 0.1
gen_kfold(DTR, LTR, pca_value, pi_value, tied_mvg, "Multivariate Gaussian classifier with tied covariance")

PCA = 7, &pi; = 0.5

In [ ]:
pca_value = 7
pi_value = 0.5
gen_kfold(DTR, LTR, pca_value, pi_value, tied_mvg, "Multivariate Gaussian classifier with tied covariance")

PCA = 7, &pi; = 0.9

In [ ]:
pca_value = 7
pi_value = 0.9
gen_kfold(DTR, LTR, pca_value, pi_value, tied_mvg, "Multivariate Gaussian classifier with tied covariance")

PCA = 6, &pi; = 0.1

In [ ]:
pca_value = 6
pi_value = 0.1
gen_kfold(DTR, LTR, pca_value, pi_value, tied_mvg, "Multivariate Gaussian classifier with tied covariance")

PCA = 6, &pi; = 0.5

In [ ]:
pca_value = 6
pi_value = 0.5
gen_kfold(DTR, LTR, pca_value, pi_value, tied_mvg, "Multivariate Gaussian classifier with tied covariance")

PCA = 6, &pi; = 0.9

In [ ]:
pca_value = 6
pi_value = 0.9
gen_kfold(DTR, LTR, pca_value, pi_value, tied_mvg, "Multivariate Gaussian classifier with tied covariance")

PCA = 5, &pi; = 0.1

In [ ]:
pca_value = 5
pi_value = 0.1
gen_kfold(DTR, LTR, pca_value, pi_value, tied_mvg, "Multivariate Gaussian classifier with tied covariance")

PCA = 5, &pi; = 0.5

In [ ]:
pca_value = 5
pi_value = 0.5
gen_kfold(DTR, LTR, pca_value, pi_value, tied_mvg, "Multivariate Gaussian classifier with tied covariance")

PCA = 5, &pi; = 0.9

In [ ]:
pca_value = 5
pi_value = 0.9
gen_kfold(DTR, LTR, pca_value, pi_value, tied_mvg, "Multivariate Gaussian classifier with tied covariance")

### 2.1.3 - Naive Bayes without tied covariance

In [ ]:
# ----- Naive Bayes without tied covariance -----
def naive_bayes (DTR, LTR, DTE, LTE):
    # Compute mean and covariance for each class
    D0 = DTR[:, LTR==0]
    D1 = DTR[:, LTR==1]
    mu0, cov0 = row_to_column(compute_mean(D0)), compute_covariance(D0)*numpy.identity(DTR.shape[0])
    mu1, cov1 = row_to_column(compute_mean(D1)), compute_covariance(D1)*numpy.identity(DTR.shape[0])

    # Compute likelihoods
    S = numpy.empty((2,DTE.shape[1]))
    for i in range(DTE.shape[1]):
        S[0,i] = numpy.exp(logpdf_GAU_ND(row_to_column(DTE[:,i]), mu0, cov0))
        S[1,i] = numpy.exp(logpdf_GAU_ND(row_to_column(DTE[:,i]), mu1, cov1))

    # Compute log-likelihood ratio and use it to classify samples
    llr = numpy.log(S[1,:] / S[0,:])
    predicted_labels = numpy.log(S).argmax(axis=0)
    wrong_predictions = len(LTE) - numpy.array(predicted_labels == LTE).sum()

    return wrong_predictions, llr

No PCA, &pi; = 0.1

In [ ]:
pca_value = 0
pi_value = 0.1
gen_kfold(DTR, LTR, pca_value, pi_value, naive_bayes, "Naive Bayes without tied covariance")

No PCA, &pi; = 0.5

In [ ]:
pca_value = 0
pi_value = 0.5
gen_kfold(DTR, LTR, pca_value, pi_value, naive_bayes, "Naive Bayes without tied covariance")

No PCA, &pi; = 0.9

In [ ]:
pca_value = 0
pi_value = 0.9
gen_kfold(DTR, LTR, pca_value, pi_value, naive_bayes, "Naive Bayes without tied covariance")

PCA = 9, &pi; = 0.1

In [ ]:
pca_value = 9
pi_value = 0.1
gen_kfold(DTR, LTR, pca_value, pi_value, naive_bayes, "Naive Bayes without tied covariance")

PCA = 9, &pi; = 0.5

In [ ]:
pca_value = 9
pi_value = 0.5
gen_kfold(DTR, LTR, pca_value, pi_value, naive_bayes, "Naive Bayes without tied covariance")

PCA = 9, &pi; = 0.9

In [ ]:
pca_value = 9
pi_value = 0.9
gen_kfold(DTR, LTR, pca_value, pi_value, naive_bayes, "Naive Bayes without tied covariance")

PCA = 8, &pi; = 0.1

In [ ]:
pca_value = 8
pi_value = 0.1
gen_kfold(DTR, LTR, pca_value, pi_value, naive_bayes, "Naive Bayes without tied covariance")

PCA = 8, &pi; = 0.5

In [ ]:
pca_value = 8
pi_value = 0.5
gen_kfold(DTR, LTR, pca_value, pi_value, naive_bayes, "Naive Bayes without tied covariance")

PCA = 8, &pi; = 0.9

In [ ]:
pca_value = 8
pi_value = 0.9
gen_kfold(DTR, LTR, pca_value, pi_value, naive_bayes, "Naive Bayes without tied covariance")

PCA = 7, &pi; = 0.1

In [ ]:
pca_value = 7
pi_value = 0.1
gen_kfold(DTR, LTR, pca_value, pi_value, naive_bayes, "Naive Bayes without tied covariance")

PCA = 7, &pi; = 0.5

In [ ]:
pca_value = 7
pi_value = 0.5
gen_kfold(DTR, LTR, pca_value, pi_value, naive_bayes, "Naive Bayes without tied covariance")

PCA 0 7, &pi; = 0.9

In [ ]:
pca_value = 7
pi_value = 0.9
gen_kfold(DTR, LTR, pca_value, pi_value, naive_bayes, "Naive Bayes without tied covariance")

PCA = 6, &pi; = 0.1

In [ ]:
pca_value = 6
pi_value = 0.1
gen_kfold(DTR, LTR, pca_value, pi_value, naive_bayes, "Naive Bayes without tied covariance")

PCA = 6, &pi; = 0.5

In [ ]:
pca_value = 6
pi_value = 0.5
gen_kfold(DTR, LTR, pca_value, pi_value, naive_bayes, "Naive Bayes without tied covariance")

PCA = 6, &pi; = 0.9

In [ ]:
pca_value = 6
pi_value = 0.9
gen_kfold(DTR, LTR, pca_value, pi_value, naive_bayes, "Naive Bayes without tied covariance")

PCA = 5, &pi; = 0.1

In [ ]:
pca_value = 5
pi_value = 0.1
gen_kfold(DTR, LTR, pca_value, pi_value, naive_bayes, "Naive Bayes without tied covariance")

PCA = 5, &pi; = 0.5

In [ ]:
pca_value = 5
pi_value = 0.5
gen_kfold(DTR, LTR, pca_value, pi_value, naive_bayes, "Naive Bayes without tied covariance")

PCA = 5, &pi; = 0.9

In [ ]:
pca_value = 5
pi_value = 0.9
gen_kfold(DTR, LTR, pca_value, pi_value, naive_bayes, "Naive Bayes without tied covariance")

### 2.1.4 - Naive Bayes with tied covariance

In [ ]:
# ----- Naive Bayes with tied covariance -----
def tied_naive_bayes (DTR, LTR, DTE, LTE):
    # Compute mean for each class and tied covariance matrix
    D0 = DTR[:, LTR==0]
    D1 = DTR[:, LTR==1]
    mu0, cov0 = row_to_column(compute_mean(D0)), compute_covariance(D0)*numpy.identity(DTR.shape[0])
    mu1, cov1 = row_to_column(compute_mean(D1)), compute_covariance(D1)*numpy.identity(DTR.shape[0])
    tied_cov = (cov0*D0.shape[1] + cov1*D1.shape[1]) / DTR.shape[1]

    # Compute likelihoods
    S = numpy.empty((2,DTE.shape[1]))
    for i in range(DTE.shape[1]):
        S[0,i] = numpy.exp(logpdf_GAU_ND(row_to_column(DTE[:,i]), mu0, tied_cov))
        S[1,i] = numpy.exp(logpdf_GAU_ND(row_to_column(DTE[:,i]), mu1, tied_cov))
    
    # Compute log-likelihood ratio and use it to classify samples
    llr = numpy.log(S[1,:] / S[0,:])
    predicted_labels = numpy.log(S).argmax(axis=0)
    wrong_predictions = len(LTE) - numpy.array(predicted_labels == LTE).sum()

    return wrong_predictions, llr

No PCA, &pi; = 0.1

In [ ]:
pca_value = 0
pi_value = 0.1
gen_kfold(DTR, LTR, pca_value, pi_value, tied_naive_bayes, "Naive Bayes with tied covariance")

No PCA, &pi; = 0.5

In [ ]:
pca_value = 0
pi_value = 0.5
gen_kfold(DTR, LTR, pca_value, pi_value, tied_naive_bayes, "Naive Bayes with tied covariance")

No PCA, &pi; = 0.9

In [ ]:
pca_value = 0
pi_value = 0.9
gen_kfold(DTR, LTR, pca_value, pi_value, tied_naive_bayes, "Naive Bayes with tied covariance")

PCA = 9, &pi; = 0.1

In [ ]:
pca_value = 9
pi_value = 0.1
gen_kfold(DTR, LTR, pca_value, pi_value, tied_naive_bayes, "Naive Bayes with tied covariance")

PCA = 9, &pi; = 0.5

In [ ]:
pca_value = 9
pi_value = 0.5
gen_kfold(DTR, LTR, pca_value, pi_value, tied_naive_bayes, "Naive Bayes with tied covariance")

PCA = 9, &pi; = 0.9

In [ ]:
pca_value = 9
pi_value = 0.9
gen_kfold(DTR, LTR, pca_value, pi_value, tied_naive_bayes, "Naive Bayes with tied covariance")

PCA = 8, &pi; = 0.1

In [ ]:
pca_value = 8
pi_value = 0.1
gen_kfold(DTR, LTR, pca_value, pi_value, tied_naive_bayes, "Naive Bayes with tied covariance")

PCA = 8, &pi; = 0.5

In [ ]:
pca_value = 8
pi_value = 0.5
gen_kfold(DTR, LTR, pca_value, pi_value, tied_naive_bayes, "Naive Bayes with tied covariance")

PCA = 8, &pi; = 0.9

In [ ]:
pca_value = 8
pi_value = 0.9
gen_kfold(DTR, LTR, pca_value, pi_value, tied_naive_bayes, "Naive Bayes with tied covariance")

PCA = 7, &pi; = 0.1

In [ ]:
pca_value = 7
pi_value = 0.1
gen_kfold(DTR, LTR, pca_value, pi_value, tied_naive_bayes, "Naive Bayes with tied covariance")

PCA = 7, &pi; = 0.5

In [ ]:
pca_value = 7
pi_value = 0.5
gen_kfold(DTR, LTR, pca_value, pi_value, tied_naive_bayes, "Naive Bayes with tied covariance")

PCA = 7, &pi; = 0.9

In [ ]:
pca_value = 7
pi_value = 0.9
gen_kfold(DTR, LTR, pca_value, pi_value, tied_naive_bayes, "Naive Bayes with tied covariance")

PCA = 6, &pi; = 0.1

In [ ]:
pca_value = 6
pi_value = 0.1
gen_kfold(DTR, LTR, pca_value, pi_value, tied_naive_bayes, "Naive Bayes with tied covariance")

PCA = 6, &pi; = 0.5

In [ ]:
pca_value = 6
pi_value = 0.5
gen_kfold(DTR, LTR, pca_value, pi_value, tied_naive_bayes, "Naive Bayes with tied covariance")

PCA = 6, &pi; = 0.9

In [ ]:
pca_value = 6
pi_value = 0.9
gen_kfold(DTR, LTR, pca_value, pi_value, tied_naive_bayes, "Naive Bayes with tied covariance")

PCA = 5, &pi; = 0.1

In [ ]:
pca_value = 5
pi_value = 0.1
gen_kfold(DTR, LTR, pca_value, pi_value, tied_naive_bayes, "Naive Bayes with tied covariance")

PCA = 5, &pi; = 0.5

In [ ]:
pca_value = 5
pi_value = 0.5
gen_kfold(DTR, LTR, pca_value, pi_value, tied_naive_bayes, "Naive Bayes with tied covariance")

PCA = 5, &pi; = 0.9

In [ ]:
pca_value = 5
pi_value = 0.9
gen_kfold(DTR, LTR, pca_value, pi_value, tied_naive_bayes, "Naive Bayes with tied covariance")

## 2.2 - Discriminative models

### 2.2.0 - Intro

Computation of costs

In [ ]:
# ----- Compute un-normalized Detection Cost Function -----
def compute_unnormalized_DCF (pi_t, Cfn, Cfp, ll_ratios, LTE, threshold=None, is_effective=False):
    # Define threshold if not previously defined
    if threshold is None:
        if is_effective is False:
            effective_prior = (pi_t*Cfn) / (pi_t*Cfn + (1-pi_t)*Cfp)
        else:
            effective_prior = pi_t
        threshold = -numpy.log(effective_prior / (1-effective_prior))
    
    # Build confusion matrix
    predicted = (ll_ratios>threshold).astype(int)
    confusion_matrix = numpy.zeros((2,2))
    for i in range(len(LTE)):
        confusion_matrix[predicted[i], LTE[i].astype(int)] += 1
    
    # Compute DCF
    fnr = confusion_matrix[0][1] / (confusion_matrix[0][1]+confusion_matrix[1][1])
    fpr = confusion_matrix[1][0] / (confusion_matrix[0][0]+confusion_matrix[1][0])
    dcf = pi_t*Cfn*fnr + (1-pi_t)*Cfp*fpr

    return dcf


# ----- Compute normalized Detection Cost Function -----
def compute_normalized_DCF (pi_t, Cfn, Cfp, DCFu):
    dummy_costs = numpy.array([pi_t*Cfn, (1-pi_t)*Cfp])
    index = numpy.argmin(dummy_costs)
    dcf = DCFu / dummy_costs[index]
    return dcf


# ----- Compute actual Detection Cost Function -----
def compute_actual_DCF (pi_t, Cfn, Cfp, ll_ratios, LTE, is_effective):
    dcfu = compute_unnormalized_DCF(pi_t, Cfn, Cfp, ll_ratios, LTE, None, is_effective)
    dcf = compute_normalized_DCF(pi_t, Cfn, Cfp, dcfu)
    return dcf


# ----- Compute minimum Detection Cost Function -----
def compute_min_DCF (pi_t, Cfn, Cfp, ll_ratios, LTE):
    dcf_collection = numpy.zeros(ll_ratios.shape)
    sorted = numpy.sort(ll_ratios)

    for i in range(len(ll_ratios)):
        threshold = sorted[i]
        unnormalized = compute_unnormalized_DCF(pi_t, Cfn, Cfp, ll_ratios, LTE, threshold)
        dcf_collection[i] = compute_normalized_DCF(pi_t, Cfn, Cfp, unnormalized)
    return numpy.min(dcf_collection)

Utility

In [ ]:
# ----- Logistic Regression objective -----
def lr_obj_wrap (DTR, LTR, lambda_value, pi_value):
    def lr_obj (v):
        w,b = v[0:-1], v[-1]
        N = DTR.shape[1]
        term1 = lambda_value/2 * numpy.linalg.norm(w)**2
        term2 = 0
        term3 = 0

        nt = DTR[:,LTR==1].shape[1]
        nf = N-nt

        for i in range(N):
            ci = LTR[i]
            zi = 2*ci-1
            xi = DTR[:,i]
            internal_sum = b + numpy.dot(w.T, xi)
            internal_prod = -numpy.dot(zi, internal_sum)
            if LTR[i]==0: term3 += numpy.logaddexp(0, internal_prod)
            else: term2 += numpy.logaddexp(0, internal_prod)
        loss = term1 + (pi_value/nt)*term2 + ((1-pi_value)/nf)*term3
        return loss
    return lr_obj


# ----- Compute Logistic Regression scores -----
def lr_compute_scores (DTE, v):
    s = []
    w,b = v[0:-1], v[-1]
    for i in range(DTE.shape[1]):
        xt = DTE[:,i]
        s.append(b + numpy.dot(w.T, xt))
    return numpy.array(s)


# ----- Numerical optimization -----
def numerical_optimization (function, x0, grad=None):
    if grad is None: x,f,_=scipy.optimize.fmin_l_bfgs_b(function,x0,fprime=numpy.gradient(function))
    else: x,_,_=scipy.optimize.fmin_l_bfgs_b(function,x0,fprime=grad)
    return x


# ----- Logistic Regression gradient -----
def lr_compute_gradient (DTR, LTR, lambda_value, pi_value):
    z = numpy.empty((LTR.shape[0]))
    z = 2*LTR-1

    def gradient (v):
        w,b = v[0:-1], v[-1]
        term1 = lambda_value * w
        term2 = 0
        term3 = 0
        
        nt = DTR[:, LTR == 1].shape[1]
        nf = DTR.shape[1]-nt
        
        for i in range(DTR.shape[1]):
            S = numpy.dot(w.T, DTR[:,i]) + b
            zi_si = numpy.dot(z[i], S)
            if LTR[i]==1: term2 += numpy.dot(numpy.exp(-zi_si),(numpy.dot(-z[i],DTR[:,i])))/(1+numpy.exp(-zi_si))
            else: term3 += numpy.dot(numpy.exp(-zi_si),(numpy.dot(-z[i],DTR[:,i])))/(1+numpy.exp(-zi_si))
        dw = term1 + (pi_value/nt)*term2 + (1-pi_value)/(nf)*term3

        term1 = 0           
        term2 = 0
        for i in range(DTR.shape[1]):
            S=numpy.dot(w.T,DTR[:,i])+b
            zi_si = numpy.dot(z[i], S)
            if LTR[i] == 1: term1 += (numpy.exp(-zi_si) * (-z[i]))/(1+numpy.exp(-zi_si))
            else: term2 += (numpy.exp(-zi_si) * (-z[i]))/(1+numpy.exp(-zi_si))
        db = (pi_value/nt)*term1 + (1-pi_value)/(nf)*term2

        return numpy.hstack((dw, db))
    return gradient

K-Fold implementation

In [ ]:
# ----- Global variables -----
K = 5
Cfn = 1
Cfp = 10
discriminative_filename = "output/04_Training/Discriminative.txt"


# ----- Compare classifiers using K-Fold -----
def dis_kfold (D, L, pca_value, pi_value, lam_value, function, name):
    N = int(D.shape[1]/K)
    
    wrong_predictions = 0
    numpy.random.seed(2)
    ll_ratios = []
    labels = []
    indexes = numpy.random.permutation(D.shape[1])

    for i in range(K):
        # Select which subset to use for evaluation
        idxTest = indexes[i*N:(i+1)*N]
        if i>0: idxTrainLeft = indexes[0:i*N]
        elif (i+1)<K: idxTrainRight = indexes[(i+1)*N:]
        if i==0: idxTrain = idxTrainRight
        elif (i+1)==K: idxTrain = idxTrainLeft
        else: idxTrain = numpy.hstack([idxTrainLeft, idxTrainRight])
        DTR,LTR = D[:,idxTrain], L[idxTrain]
        DTE,LTE = D[:,idxTest], L[idxTest]

        # Apply PCA if necessary
        if pca_value!=0:
            P = apply_pca(DTR, LTR, pca_value)
            DTR,DTE = numpy.dot(P.T, DTR), numpy.dot(P.T, DTE)

        # Apply classifier
        wrong, scores = function(DTR, LTR, DTE, LTE, lam_value, pi_value)
        wrong_predictions += wrong
        ll_ratios.append(scores)
        labels.append(LTE)

    # Evaluate accuracy, error rate, and minDCF
    error_rate = wrong_predictions / D.shape[1]
    accuracy = 1 - error_rate
    cost = compute_min_DCF(pi_value, Cfn, Cfp, numpy.hstack(ll_ratios), numpy.hstack(labels))
    
    # Print results
    print("%s" % (name))
    if (pca_value==0): print("  No PCA, pi = %.2f, lambda = %.7f" % (pi_value, lam_value))
    else: print("  PCA = %d, pi = %.2f, lambda = %.7f" % (pca_value, pi_value, lam_value))
    print("  Accuracy: %.3f%%" % (100.0*accuracy))
    print("  Error rate: %.3f%%" % (100.0*error_rate))
    print("  min DCF: %.3f" % (cost))
    print("\n")

    # Save results to file
    with open(discriminative_filename, "a") as file:
        file.write("%s\n" % (name))
        if (pca_value==0): file.write("  No PCA, pi = %.2f, lambda = %.7f\n" % (pi_value, lam_value))
        else: file.write("  PCA = %d, pi = %.2f, lambda = %.7f\n" % (pca_value, pi_value, lam_value))
        file.write("  Accuracy: %.3f%%\n" % (100.0*accuracy))
        file.write("  Error rate: %.3f%%\n" % (100.0*error_rate))
        file.write("  min DCF: %.3f\n" % (cost))
        file.write("\n\n")

### 2.2.1 - Linear Logistic Regression

In [ ]:
# ----- Linear Logistic Regression -----
def linear_logistic_regression (DTR, LTR, DTE, LTE, lam, pi):
    x0 = numpy.zeros(DTR.shape[0]+1)
    x = numerical_optimization(lr_obj_wrap(DTR, LTR, lam, pi), x0, lr_compute_gradient(DTR, LTR, lam, pi))
    
    # Compute scores
    scores = lr_compute_scores(DTE, x)
    predicted_labels = predict_labels(scores, 0)
    wrong_predictions = count_mispredictions(predicted_labels, LTE)

    return wrong_predictions, scores

No PCA, &pi; = 0.1, &lambda; = 1e-6

In [ ]:
pca_value = 0
pi_value = 0.1
lam_value = 1e-6
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

No PCA, &pi; = 0.1, &lambda; = 1e-5

In [ ]:
pca_value = 0
pi_value = 0.1
lam_value = 1e-5
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

No PCA, &pi; = 0.1, &lambda; = 1e-4

In [ ]:
pca_value = 0
pi_value = 0.1
lam_value = 1e-4
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

No PCA, &pi; = 0.1, &lambda; = 1e-3

In [ ]:
pca_value = 0
pi_value = 0.1
lam_value = 1e-3
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

No PCA, &pi; = 0.1, &lambda; = 1e-2

In [ ]:
pca_value = 0
pi_value = 0.1
lam_value = 1e-2
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

No PCA, &pi; = 0.1, &lambda; = 1e-1

In [ ]:
pca_value = 0
pi_value = 0.1
lam_value = 1e-1
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

No PCA, &pi; = 0.1, &lambda; = 1.0

In [ ]:
pca_value = 0
pi_value = 0.1
lam_value = 1.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

No PCA, &pi; = 0.1, &lambda; = 10.0

In [ ]:
pca_value = 0
pi_value = 0.1
lam_value = 10.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

No PCA, &pi; = 0.1, &lambda; = 100.0

In [ ]:
pca_value = 0
pi_value = 0.1
lam_value = 100.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

No PCA, &pi; = 0.5, &lambda; = 1e-6

In [ ]:
pca_value = 0
pi_value = 0.5
lam_value = 1e-6
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

No PCA, &pi; = 0.5, &lambda; = 1e-5

In [ ]:
pca_value = 0
pi_value = 0.5
lam_value = 1e-5
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

No PCA, &pi; = 0.5, &lambda; = 1e-4

In [ ]:
pca_value = 0
pi_value = 0.5
lam_value = 1e-4
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

No PCA, &pi; = 0.5, &lambda; = 1e-3

In [ ]:
pca_value = 0
pi_value = 0.5
lam_value = 1e-3
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

No PCA, &pi; = 0.5, &lambda; = 1e-2

In [ ]:
pca_value = 0
pi_value = 0.5
lam_value = 1e-2
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

No PCA, &pi; = 0.5, &lambda; = 1e-1

In [ ]:
pca_value = 0
pi_value = 0.5
lam_value = 1e-1
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

No PCA, &pi; = 0.5, &lambda; = 1.0

In [ ]:
pca_value = 0
pi_value = 0.5
lam_value = 1.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

No PCA, &pi; = 0.5, &lambda; = 10.0

In [ ]:
pca_value = 0
pi_value = 0.5
lam_value = 10.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

No PCA, &pi; = 0.5, &lambda; = 100.0

In [ ]:
pca_value = 0
pi_value = 0.5
lam_value = 100.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

No PCA, &pi; = 0.9, &lambda; = 1e-6

In [ ]:
pca_value = 0
pi_value = 0.9
lam_value = 1e-6
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

No PCA, &pi; = 0.9, &lambda; = 1e-5

In [ ]:
pca_value = 0
pi_value = 0.9
lam_value = 1e-5
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

No PCA, &pi; = 0.9, &lambda; = 1e-4

In [ ]:
pca_value = 0
pi_value = 0.9
lam_value = 1e-4
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

No PCA, &pi; = 0.9, &lambda; = 1e-3

In [ ]:
pca_value = 0
pi_value = 0.9
lam_value = 1e-3
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

No PCA, &pi; = 0.9, &lambda; = 1e-2

In [ ]:
pca_value = 0
pi_value = 0.9
lam_value = 1e-2
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

No PCA, &pi; = 0.9, &lambda; = 1e-1

In [ ]:
pca_value = 0
pi_value = 0.9
lam_value = 1e-1
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

No PCA, &pi; = 0.9, &lambda; = 1.0

In [ ]:
pca_value = 0
pi_value = 0.9
lam_value = 1.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

No PCA, &pi; = 0.9, &lambda; = 10.0

In [ ]:
pca_value = 0
pi_value = 0.9
lam_value = 10.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

No PCA, &pi; = 0.9, &lambda; = 100.0

In [ ]:
pca_value = 0
pi_value = 0.9
lam_value = 100.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 9, &pi; = 0.1, &lambda; = 1e-6

In [ ]:
pca_value = 9
pi_value = 0.1
lam_value = 1e-6
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 9, &pi; = 0.1, &lambda; = 1e-5

In [ ]:
pca_value = 9
pi_value = 0.1
lam_value = 1e-5
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 9, &pi; = 0.1, &lambda; = 1e-4

In [ ]:
pca_value = 9
pi_value = 0.1
lam_value = 1e-4
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 9, &pi; = 0.1, &lambda; = 1e-3

In [ ]:
pca_value = 9
pi_value = 0.1
lam_value = 1e-3
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 9, &pi; = 0.1, &lambda; = 1e-2

In [ ]:
pca_value = 9
pi_value = 0.1
lam_value = 1e-2
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 9, &pi; = 0.1, &lambda; = 1e-1

In [ ]:
pca_value = 9
pi_value = 0.1
lam_value = 1e-1
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 9, &pi; = 0.1, &lambda; = 1.0

In [ ]:
pca_value = 9
pi_value = 0.1
lam_value = 1.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 9, &pi; = 0.1, &lambda; = 10.0

In [ ]:
pca_value = 9
pi_value = 0.1
lam_value = 10.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 9, &pi; = 0.1, &lambda; = 100.0

In [ ]:
pca_value = 9
pi_value = 0.1
lam_value = 100.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 9, &pi; = 0.5, &lambda; = 1e-6

In [ ]:
pca_value = 9
pi_value = 0.5
lam_value = 1e-6
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 9, &pi; = 0.5, &lambda; = 1e-5

In [ ]:
pca_value = 9
pi_value = 0.5
lam_value = 1e-5
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 9, &pi; = 0.5, &lambda; = 1e-4

In [ ]:
pca_value = 9
pi_value = 0.5
lam_value = 1e-4
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 9, &pi; = 0.5, &lambda; = 1e-3

In [ ]:
pca_value = 9
pi_value = 0.5
lam_value = 1e-3
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 9, &pi; = 0.5, &lambda; = 1e-2

In [ ]:
pca_value = 9
pi_value = 0.5
lam_value = 1e-2
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 9, &pi; = 0.5, &lambda; = 1e-1

In [ ]:
pca_value = 9
pi_value = 0.5
lam_value = 1e-1
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 9, &pi; = 0.5, &lambda; = 1.0

In [ ]:
pca_value = 9
pi_value = 0.5
lam_value = 1.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 9, &pi; = 0.5, &lambda; = 10.0

In [ ]:
pca_value = 9
pi_value = 0.5
lam_value = 10.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 9, &pi; = 0.5, &lambda; = 100.0

In [ ]:
pca_value = 9
pi_value = 0.5
lam_value = 100.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 9, &pi; = 0.9, &lambda; = 1e-6

In [ ]:
pca_value = 9
pi_value = 0.9
lam_value = 1e-6
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 9, &pi; = 0.9, &lambda; = 1e-5

In [ ]:
pca_value = 9
pi_value = 0.9
lam_value = 1e-5
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 9, &pi; = 0.9, &lambda; = 1e-4

In [ ]:
pca_value = 9
pi_value = 0.9
lam_value = 1e-4
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 9, &pi; = 0.9, &lambda; = 1e-3

In [ ]:
pca_value = 9
pi_value = 0.9
lam_value = 1e-3
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 9, &pi; = 0.9, &lambda; = 1e-2

In [ ]:
pca_value = 9
pi_value = 0.9
lam_value = 1e-2
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 9, &pi; = 0.9, &lambda; = 1e-1

In [ ]:
pca_value = 9
pi_value = 0.9
lam_value = 1e-1
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 9, &pi; = 0.9, &lambda; = 1.0

In [ ]:
pca_value = 9
pi_value = 0.9
lam_value = 1.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 9, &pi; = 0.9, &lambda; = 10.0

In [ ]:
pca_value = 9
pi_value = 0.9
lam_value = 10.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 9, &pi; = 0.9, &lambda; = 100.0

In [ ]:
pca_value = 9
pi_value = 0.9
lam_value = 100.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 8, &pi; = 0.1, &lambda; = 1e-6

In [ ]:
pca_value = 8
pi_value = 0.1
lam_value = 1e-6
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 8, &pi; = 0.1, &lambda; = 1e-5

In [ ]:
pca_value = 8
pi_value = 0.1
lam_value = 1e-5
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 8, &pi; = 0.1, &lambda; = 1e-4

In [ ]:
pca_value = 8
pi_value = 0.1
lam_value = 1e-4
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 8, &pi; = 0.1, &lambda; = 1e-3

In [ ]:
pca_value = 8
pi_value = 0.1
lam_value = 1e-3
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 8, &pi; = 0.1, &lambda; = 1e-2

In [ ]:
pca_value = 8
pi_value = 0.1
lam_value = 1e-2
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 8, &pi; = 0.1, &lambda; = 1e-1

In [ ]:
pca_value = 8
pi_value = 0.1
lam_value = 1e-1
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 8, &pi; = 0.1, &lambda; = 1.0

In [ ]:
pca_value = 8
pi_value = 0.1
lam_value = 1.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 8, &pi; = 0.1, &lambda; = 10.0

In [ ]:
pca_value = 8
pi_value = 0.1
lam_value = 10.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 8, &pi; = 0.1, &lambda; = 100.0

In [ ]:
pca_value = 8
pi_value = 0.1
lam_value = 100.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 8, &pi; = 0.5, &lambda; = 1e-6

In [ ]:
pca_value = 8
pi_value = 0.5
lam_value = 1e-6
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 8, &pi; = 0.5, &lambda; = 1e-5

In [ ]:
pca_value = 8
pi_value = 0.5
lam_value = 1e-5
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 8, &pi; = 0.5, &lambda; = 1e-4

In [ ]:
pca_value = 8
pi_value = 0.5
lam_value = 1e-4
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 8, &pi; = 0.5, &lambda; = 1e-3

In [ ]:
pca_value = 8
pi_value = 0.5
lam_value = 1e-3
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 8, &pi; = 0.5, &lambda; = 1e-2

In [ ]:
pca_value = 8
pi_value = 0.5
lam_value = 1e-2
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 8, &pi; = 0.5, &lambda; = 1e-1

In [ ]:
pca_value = 8
pi_value = 0.5
lam_value = 1e-1
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 8, &pi; = 0.5, &lambda; = 1.0

In [ ]:
pca_value = 8
pi_value = 0.5
lam_value = 1.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 8, &pi; = 0.5, &lambda; = 10.0

In [ ]:
pca_value = 8
pi_value = 0.5
lam_value = 10.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 8, &pi; = 0.5, &lambda; = 100.0

In [ ]:
pca_value = 8
pi_value = 0.5
lam_value = 100.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 8, &pi; = 0.9, &lambda; = 1e-6

In [ ]:
pca_value = 8
pi_value = 0.9
lam_value = 1e-6
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 8, &pi; = 0.9, &lambda; = 1e-5

In [ ]:
pca_value = 8
pi_value = 0.9
lam_value = 1e-5
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 8, &pi; = 0.9, &lambda; = 1e-4

In [ ]:
pca_value = 8
pi_value = 0.9
lam_value = 1e-4
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 8, &pi; = 0.9, &lambda; = 1e-3

In [ ]:
pca_value = 8
pi_value = 0.9
lam_value = 1e-3
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 8, &pi; = 0.9, &lambda; = 1e-2

In [ ]:
pca_value = 8
pi_value = 0.9
lam_value = 1e-2
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 8, &pi; = 0.9, &lambda; = 1e-1

In [ ]:
pca_value = 8
pi_value = 0.9
lam_value = 1e-1
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 8, &pi; = 0.9, &lambda; = 1.0

In [ ]:
pca_value = 8
pi_value = 0.9
lam_value = 1.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 8, &pi; = 0.9, &lambda; = 10.0

In [ ]:
pca_value = 8
pi_value = 0.9
lam_value = 10.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 8, &pi; = 0.9, &lambda; = 100.0

In [ ]:
pca_value = 8
pi_value = 0.9
lam_value = 100.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 7, &pi; = 0.1, &lambda; = 1e-6

In [ ]:
pca_value = 7
pi_value = 0.1
lam_value = 1e-6
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 7, &pi; = 0.1, &lambda; = 1e-5

In [ ]:
pca_value = 7
pi_value = 0.1
lam_value = 1e-5
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 7, &pi; = 0.1, &lambda; = 1e-4

In [ ]:
pca_value = 7
pi_value = 0.1
lam_value = 1e-4
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 7, &pi; = 0.1, &lambda; = 1e-3

In [ ]:
pca_value = 7
pi_value = 0.1
lam_value = 1e-3
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 7, &pi; = 0.1, &lambda; = 1e-2

In [ ]:
pca_value = 7
pi_value = 0.1
lam_value = 1e-2
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 7, &pi; = 0.1, &lambda; = 1e-1

In [ ]:
pca_value = 7
pi_value = 0.1
lam_value = 1e-1
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 7, &pi; = 0.1, &lambda; = 1.0

In [ ]:
pca_value = 7
pi_value = 0.1
lam_value = 1.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 7, &pi; = 0.1, &lambda; = 10.0

In [ ]:
pca_value = 7
pi_value = 0.1
lam_value = 10.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 7, &pi; = 0.1, &lambda; = 100.0

In [ ]:
pca_value = 7
pi_value = 0.1
lam_value = 100.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 7, &pi; = 0.5, &lambda; = 1e-6

In [ ]:
pca_value = 7
pi_value = 0.5
lam_value = 1e-6
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 7, &pi; = 0.5, &lambda; = 1e-5

In [ ]:
pca_value = 7
pi_value = 0.5
lam_value = 1e-5
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 7, &pi; = 0.5, &lambda; = 1e-4

In [ ]:
pca_value = 7
pi_value = 0.5
lam_value = 1e-4
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 7, &pi; = 0.5, &lambda; = 1e-3

In [ ]:
pca_value = 7
pi_value = 0.5
lam_value = 1e-3
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 7, &pi; = 0.5, &lambda; = 1e-2

In [ ]:
pca_value = 7
pi_value = 0.5
lam_value = 1e-2
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 7, &pi; = 0.5, &lambda; = 1e-1

In [ ]:
pca_value = 7
pi_value = 0.5
lam_value = 1e-1
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 7, &pi; = 0.5, &lambda; = 1.0

In [ ]:
pca_value = 7
pi_value = 0.5
lam_value = 1.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 7, &pi; = 0.5, &lambda; = 10.0

In [ ]:
pca_value = 7
pi_value = 0.5
lam_value = 10.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 7, &pi; = 0.5, &lambda; = 100.0

In [ ]:
pca_value = 7
pi_value = 0.5
lam_value = 100.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 7, &pi; = 0.9, &lambda; = 1e-6

In [ ]:
pca_value = 7
pi_value = 0.9
lam_value = 1e-6
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 7, &pi; = 0.9, &lambda; = 1e-5

In [ ]:
pca_value = 7
pi_value = 0.9
lam_value = 1e-5
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 7, &pi; = 0.9, &lambda; = 1e-4

In [ ]:
pca_value = 7
pi_value = 0.9
lam_value = 1e-4
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 7, &pi; = 0.9, &lambda; = 1e-3

In [ ]:
pca_value = 7
pi_value = 0.9
lam_value = 1e-3
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 7, &pi; = 0.9, &lambda; = 1e-2

In [ ]:
pca_value = 7
pi_value = 0.9
lam_value = 1e-2
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 7, &pi; = 0.9, &lambda; = 1e-1

In [ ]:
pca_value = 7
pi_value = 0.9
lam_value = 1e-1
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 7, &pi; = 0.9, &lambda; = 1.0

In [ ]:
pca_value = 7
pi_value = 0.9
lam_value = 1.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 7, &pi; = 0.9, &lambda; = 10.0

In [ ]:
pca_value = 7
pi_value = 0.9
lam_value = 10.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 7, &pi; = 0.9, &lambda; = 100.0

In [ ]:
pca_value = 7
pi_value = 0.9
lam_value = 100.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 6, &pi; = 0.1, &lambda; = 1e-6

In [ ]:
pca_value = 6
pi_value = 0.1
lam_value = 1e-6
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 6, &pi; = 0.1, &lambda; = 1e-5

In [ ]:
pca_value = 6
pi_value = 0.1
lam_value = 1e-5
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 6, &pi; = 0.1, &lambda; = 1e-4

In [ ]:
pca_value = 6
pi_value = 0.1
lam_value = 1e-4
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 6, &pi; = 0.1, &lambda; = 1e-3

In [ ]:
pca_value = 6
pi_value = 0.1
lam_value = 1e-3
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 6, &pi; = 0.1, &lambda; = 1e-2

In [ ]:
pca_value = 6
pi_value = 0.1
lam_value = 1e-2
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 6, &pi; = 0.1, &lambda; = 1e-1

In [ ]:
pca_value = 6
pi_value = 0.1
lam_value = 1e-1
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 6, &pi; = 0.1, &lambda; = 1.0

In [ ]:
pca_value = 6
pi_value = 0.1
lam_value = 1.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 6, &pi; = 0.1, &lambda; = 10.0

In [ ]:
pca_value = 6
pi_value = 0.1
lam_value = 10.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 6, &pi; = 0.1, &lambda; = 100.0

In [ ]:
pca_value = 6
pi_value = 0.1
lam_value = 100.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 6, &pi; = 0.5, &lambda; = 1e-6

In [ ]:
pca_value = 6
pi_value = 0.5
lam_value = 1e-6
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 6, &pi; = 0.5, &lambda; = 1e-5

In [ ]:
pca_value = 6
pi_value = 0.5
lam_value = 1e-5
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 6, &pi; = 0.5, &lambda; = 1e-4

In [ ]:
pca_value = 6
pi_value = 0.5
lam_value = 1e-4
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 6, &pi; = 0.5, &lambda; = 1e-3

In [ ]:
pca_value = 6
pi_value = 0.5
lam_value = 1e-3
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 6, &pi; = 0.5, &lambda; = 1e-2

In [ ]:
pca_value = 6
pi_value = 0.5
lam_value = 1e-2
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 6, &pi; = 0.5, &lambda; = 1e-1

In [ ]:
pca_value = 6
pi_value = 0.5
lam_value = 1e-1
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 6, &pi; = 0.5, &lambda; = 1.0

In [ ]:
pca_value = 6
pi_value = 0.5
lam_value = 1.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 6, &pi; = 0.5, &lambda; = 10.0

In [ ]:
pca_value = 6
pi_value = 0.5
lam_value = 10.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 6, &pi; = 0.5, &lambda; = 100.0

In [ ]:
pca_value = 6
pi_value = 0.5
lam_value = 100.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 6, &pi; = 0.9, &lambda; = 1e-6

In [ ]:
pca_value = 6
pi_value = 0.9
lam_value = 1e-6
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 6, &pi; = 0.9, &lambda; = 1e-5

In [ ]:
pca_value = 6
pi_value = 0.9
lam_value = 1e-5
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 6, &pi; = 0.9, &lambda; = 1e-4

In [ ]:
pca_value = 6
pi_value = 0.9
lam_value = 1e-4
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 6, &pi; = 0.9, &lambda; = 1e-3

In [ ]:
pca_value = 6
pi_value = 0.9
lam_value = 1e-3
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 6, &pi; = 0.9, &lambda; = 1e-2

In [ ]:
pca_value = 6
pi_value = 0.9
lam_value = 1e-2
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 6, &pi; = 0.9, &lambda; = 1e-1

In [ ]:
pca_value = 6
pi_value = 0.9
lam_value = 1e-1
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 6, &pi; = 0.9, &lambda; = 1.0

In [ ]:
pca_value = 6
pi_value = 0.9
lam_value = 1.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 6, &pi; = 0.9, &lambda; = 10.0

In [ ]:
pca_value = 6
pi_value = 0.9
lam_value = 10.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 6, &pi; = 0.9, &lambda; = 100.0

In [ ]:
pca_value = 6
pi_value = 0.9
lam_value = 100.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 5, &pi; = 0.1, &lambda; = 1e-6

In [ ]:
pca_value = 5
pi_value = 0.1
lam_value = 1e-6
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 5, &pi; = 0.1, &lambda; = 1e-5

In [ ]:
pca_value = 5
pi_value = 0.1
lam_value = 1e-5
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 5, &pi; = 0.1, &lambda; = 1e-4

In [ ]:
pca_value = 5
pi_value = 0.1
lam_value = 1e-4
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 5, &pi; = 0.1, &lambda; = 1e-3

In [ ]:
pca_value = 5
pi_value = 0.1
lam_value = 1e-3
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 5, &pi; = 0.1, &lambda; = 1e-2

In [ ]:
pca_value = 5
pi_value = 0.1
lam_value = 1e-2
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 5, &pi; = 0.1, &lambda; = 1e-1

In [ ]:
pca_value = 5
pi_value = 0.1
lam_value = 1e-1
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 5, &pi; = 0.1, &lambda; = 1.0

In [ ]:
pca_value = 5
pi_value = 0.1
lam_value = 1.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 5, &pi; = 0.1, &lambda; = 10.0

In [ ]:
pca_value = 5
pi_value = 0.1
lam_value = 10.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 5, &pi; = 0.1, &lambda; = 100.0

In [ ]:
pca_value = 5
pi_value = 0.1
lam_value = 100.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 5, &pi; = 0.5, &lambda; = 1e-6

In [ ]:
pca_value = 5
pi_value = 0.5
lam_value = 1e-6
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 5, &pi; = 0.5, &lambda; = 1e-5

In [ ]:
pca_value = 5
pi_value = 0.5
lam_value = 1e-5
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 5, &pi; = 0.5, &lambda; = 1e-4

In [ ]:
pca_value = 5
pi_value = 0.5
lam_value = 1e-4
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 5, &pi; = 0.5, &lambda; = 1e-3

In [ ]:
pca_value = 5
pi_value = 0.5
lam_value = 1e-3
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 5, &pi; = 0.5, &lambda; = 1e-2

In [ ]:
pca_value = 5
pi_value = 0.5
lam_value = 1e-2
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 5, &pi; = 0.5, &lambda; = 1e-1

In [ ]:
pca_value = 5
pi_value = 0.5
lam_value = 1e-1
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 5, &pi; = 0.5, &lambda; = 1.0

In [ ]:
pca_value = 5
pi_value = 0.5
lam_value = 1.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 5, &pi; = 0.5, &lambda; = 10.0

In [ ]:
pca_value = 5
pi_value = 0.5
lam_value = 10.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 5, &pi; = 0.5, &lambda; = 100.0

In [ ]:
pca_value = 5
pi_value = 0.5
lam_value = 100.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 5, &pi; = 0.9, &lambda; = 1e-6

In [ ]:
pca_value = 5
pi_value = 0.9
lam_value = 1e-6
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 5, &pi; = 0.9, &lambda; = 1e-5

In [ ]:
pca_value = 5
pi_value = 0.9
lam_value = 1e-5
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 5, &pi; = 0.9, &lambda; = 1e-4

In [ ]:
pca_value = 5
pi_value = 0.9
lam_value = 1e-4
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 5, &pi; = 0.9, &lambda; = 1e-3

In [ ]:
pca_value = 5
pi_value = 0.9
lam_value = 1e-3
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 5, &pi; = 0.9, &lambda; = 1e-2

In [ ]:
pca_value = 5
pi_value = 0.9
lam_value = 1e-2
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 5, &pi; = 0.9, &lambda; = 1e-1

In [ ]:
pca_value = 5
pi_value = 0.9
lam_value = 1e-1
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 5, &pi; = 0.9, &lambda; = 1.0

In [ ]:
pca_value = 5
pi_value = 0.9
lam_value = 1.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 5, &pi; = 0.9, &lambda; = 10.0

In [ ]:
pca_value = 5
pi_value = 0.9
lam_value = 10.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

PCA = 5, &pi; = 0.9, &lambda; = 100.0

In [ ]:
pca_value = 5
pi_value = 0.9
lam_value = 100.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, linear_logistic_regression, "Linear Logistic Regression")

### 2.2.2 - Quadratic Logistic Regression

In [ ]:
# ----- Quadratic Logistic Regression -----
def quadratic_logistic_regression (DTR, LTR, DTE, LTE, lam, pi):
    DTRe = numpy.apply_along_axis(square_and_transpose, 0, DTR)
    DTEe = numpy.apply_along_axis(square_and_transpose, 0, DTE)
    phi_T = numpy.array(numpy.vstack([DTRe, DTR]))
    phi_E = numpy.array(numpy.vstack([DTEe, DTE]))
        
    x0 = numpy.zeros(phi_T.shape[0]+1)
    x = numerical_optimization(lr_obj_wrap(phi_T, LTR, lam, pi), x0, lr_compute_gradient(phi_T, LTR, lam, pi))

    # Compute scores and wrong predictions
    scores = lr_compute_scores(phi_E, x)
    predicted_labels = predict_labels(scores, 0)
    wrong_predictions = count_mispredictions(predicted_labels, LTE)

    return wrong_predictions, scores

No PCA, &pi; = 0.1, &lambda; = 1e-6

In [ ]:
pca_value = 0
pi_value = 0.1
lam_value = 1e-6
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

No PCA, &pi; = 0.1, &lambda; = 1e-5

In [ ]:
pca_value = 0
pi_value = 0.1
lam_value = 1e-5
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

No PCA, &pi; = 0.1, &lambda; = 1e-4

In [ ]:
pca_value = 0
pi_value = 0.1
lam_value = 1e-4
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

No PCA, &pi; = 0.1, &lambda; = 1e-3

In [ ]:
pca_value = 0
pi_value = 0.1
lam_value = 1e-3
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

No PCA, &pi; = 0.1, &lambda; = 1e-2

In [ ]:
pca_value = 0
pi_value = 0.1
lam_value = 1e-2
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

No PCA, &pi; = 0.1, &lambda; = 1e-1

In [ ]:
pca_value = 0
pi_value = 0.1
lam_value = 1e-1
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

No PCA, &pi; = 0.1, &lambda; = 1.0

In [ ]:
pca_value = 0
pi_value = 0.1
lam_value = 1.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

No PCA, &pi; = 0.1, &lambda; = 10.0

In [ ]:
pca_value = 0
pi_value = 0.1
lam_value = 10.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

No PCA, &pi; = 0.1, &lambda; = 100.0

In [ ]:
pca_value = 0
pi_value = 0.1
lam_value = 100.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

No PCA, &pi; = 0.5, &lambda; = 1e-6

In [ ]:
pca_value = 0
pi_value = 0.5
lam_value = 1e-6
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

No PCA, &pi; = 0.5, &lambda; = 1e-5

In [ ]:
pca_value = 0
pi_value = 0.5
lam_value = 1e-5
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

No PCA, &pi; = 0.5, &lambda; = 1e-4

In [ ]:
pca_value = 0
pi_value = 0.5
lam_value = 1e-4
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

No PCA, &pi; = 0.5, &lambda; = 1e-3

In [ ]:
pca_value = 0
pi_value = 0.5
lam_value = 1e-3
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

No PCA, &pi; = 0.5, &lambda; = 1e-2

In [ ]:
pca_value = 0
pi_value = 0.5
lam_value = 1e-2
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

No PCA, &pi; = 0.5, &lambda; = 1e-1

In [ ]:
pca_value = 0
pi_value = 0.5
lam_value = 1e-1
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

No PCA, &pi; = 0.5, &lambda; = 1.0

In [ ]:
pca_value = 0
pi_value = 0.5
lam_value = 1.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

No PCA, &pi; = 0.5, &lambda; = 10.0

In [ ]:
pca_value = 0
pi_value = 0.5
lam_value = 10.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

No PCA, &pi; = 0.5, &lambda; = 100.0

In [ ]:
pca_value = 0
pi_value = 0.5
lam_value = 100.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

No PCA, &pi; = 0.9, &lambda; = 1e-6

In [ ]:
pca_value = 0
pi_value = 0.9
lam_value = 1e-6
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

No PCA, &pi; = 0.9, &lambda; = 1e-5

In [ ]:
pca_value = 0
pi_value = 0.9
lam_value = 1e-5
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

No PCA, &pi; = 0.9, &lambda; = 1e-4

In [ ]:
pca_value = 0
pi_value = 0.9
lam_value = 1e-4
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

No PCA, &pi; = 0.9, &lambda; = 1e-3

In [ ]:
pca_value = 0
pi_value = 0.9
lam_value = 1e-3
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

No PCA, &pi; = 0.9, &lambda; = 1e-2

In [ ]:
pca_value = 0
pi_value = 0.9
lam_value = 1e-2
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

No PCA, &pi; = 0.9, &lambda; = 1e-1

In [ ]:
pca_value = 0
pi_value = 0.9
lam_value = 1e-1
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

No PCA, &pi; = 0.9, &lambda; = 1.0

In [ ]:
pca_value = 0
pi_value = 0.9
lam_value = 1.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

No PCA, &pi; = 0.9, &lambda; = 10.0

In [ ]:
pca_value = 0
pi_value = 0.9
lam_value = 10.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

No PCA, &pi; = 0.9, &lambda; = 100.0

In [ ]:
pca_value = 0
pi_value = 0.9
lam_value = 100.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 9, &pi; = 0.1, &lambda; = 1e-6

In [ ]:
pca_value = 9
pi_value = 0.1
lam_value = 1e-6
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 9, &pi; = 0.1, &lambda; = 1e-5

In [ ]:
pca_value = 9
pi_value = 0.1
lam_value = 1e-5
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 9, &pi; = 0.1, &lambda; = 1e-4

In [ ]:
pca_value = 9
pi_value = 0.1
lam_value = 1e-4
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 9, &pi; = 0.1, &lambda; = 1e-3

In [ ]:
pca_value = 9
pi_value = 0.1
lam_value = 1e-3
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 9, &pi; = 0.1, &lambda; = 1e-2

In [ ]:
pca_value = 9
pi_value = 0.1
lam_value = 1e-2
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 9, &pi; = 0.1, &lambda; = 1e-1

In [ ]:
pca_value = 9
pi_value = 0.1
lam_value = 1e-1
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 9, &pi; = 0.1, &lambda; = 1.0

In [ ]:
pca_value = 9
pi_value = 0.1
lam_value = 1.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 9, &pi; = 0.1, &lambda; = 10.0

In [ ]:
pca_value = 9
pi_value = 0.1
lam_value = 10.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 9, &pi; = 0.1, &lambda; = 100.0

In [ ]:
pca_value = 9
pi_value = 0.1
lam_value = 100.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 9, &pi; = 0.5, &lambda; = 1e-6

In [ ]:
pca_value = 9
pi_value = 0.5
lam_value = 1e-6
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 9, &pi; = 0.5, &lambda; = 1e-5

In [ ]:
pca_value = 9
pi_value = 0.5
lam_value = 1e-5
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 9, &pi; = 0.5, &lambda; = 1e-4

In [ ]:
pca_value = 9
pi_value = 0.5
lam_value = 1e-4
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 9, &pi; = 0.5, &lambda; = 1e-3

In [ ]:
pca_value = 9
pi_value = 0.5
lam_value = 1e-3
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 9, &pi; = 0.5, &lambda; = 1e-2

In [ ]:
pca_value = 9
pi_value = 0.5
lam_value = 1e-2
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 9, &pi; = 0.5, &lambda; = 1e-1

In [ ]:
pca_value = 9
pi_value = 0.5
lam_value = 1e-1
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 9, &pi; = 0.5, &lambda; = 1.0

In [ ]:
pca_value = 9
pi_value = 0.5
lam_value = 1.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 9, &pi; = 0.5, &lambda; = 10.0

In [ ]:
pca_value = 9
pi_value = 0.5
lam_value = 10.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 9, &pi; = 0.5, &lambda; = 100.0

In [ ]:
pca_value = 9
pi_value = 0.5
lam_value = 100.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 9, &pi; = 0.9, &lambda; = 1e-6

In [ ]:
pca_value = 9
pi_value = 0.9
lam_value = 1e-6
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 9, &pi; = 0.9, &lambda; = 1e-5

In [ ]:
pca_value = 9
pi_value = 0.9
lam_value = 1e-5
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 9, &pi; = 0.9, &lambda; = 1e-4

In [ ]:
pca_value = 9
pi_value = 0.9
lam_value = 1e-4
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 9, &pi; = 0.9, &lambda; = 1e-3

In [ ]:
pca_value = 9
pi_value = 0.9
lam_value = 1e-3
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 9, &pi; = 0.9, &lambda; = 1e-2

In [ ]:
pca_value = 9
pi_value = 0.9
lam_value = 1e-2
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 9, &pi; = 0.9, &lambda; = 1e-1

In [ ]:
pca_value = 9
pi_value = 0.9
lam_value = 1e-1
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 9, &pi; = 0.9, &lambda; = 1.0

In [ ]:
pca_value = 9
pi_value = 0.9
lam_value = 1.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 9, &pi; = 0.9, &lambda; = 10.0

In [ ]:
pca_value = 9
pi_value = 0.9
lam_value = 10.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 9, &pi; = 0.9, &lambda; = 100.0

In [ ]:
pca_value = 9
pi_value = 0.9
lam_value = 100.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 8, &pi; = 0.1, &lambda; = 1e-6

In [ ]:
pca_value = 8
pi_value = 0.1
lam_value = 1e-6
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 8, &pi; = 0.1, &lambda; = 1e-5

In [ ]:
pca_value = 8
pi_value = 0.1
lam_value = 1e-5
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 8, &pi; = 0.1, &lambda; = 1e-4

In [ ]:
pca_value = 8
pi_value = 0.1
lam_value = 1e-4
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 8, &pi; = 0.1, &lambda; = 1e-3

In [ ]:
pca_value = 8
pi_value = 0.1
lam_value = 1e-3
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 8, &pi; = 0.1, &lambda; = 1e-2

In [ ]:
pca_value = 8
pi_value = 0.1
lam_value = 1e-2
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 8, &pi; = 0.1, &lambda; = 1e-1

In [ ]:
pca_value = 8
pi_value = 0.1
lam_value = 1e-1
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 8, &pi; = 0.1, &lambda; = 1.0

In [ ]:
pca_value = 8
pi_value = 0.1
lam_value = 1.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 8, &pi; = 0.1, &lambda; = 10.0

In [ ]:
pca_value = 8
pi_value = 0.1
lam_value = 10.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 8, &pi; = 0.1, &lambda; = 100.0

In [ ]:
pca_value = 8
pi_value = 0.1
lam_value = 100.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 8, &pi; = 0.5, &lambda; = 1e-6

In [ ]:
pca_value = 8
pi_value = 0.5
lam_value = 1e-6
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 8, &pi; = 0.5, &lambda; = 1e-5

In [ ]:
pca_value = 8
pi_value = 0.5
lam_value = 1e-5
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 8, &pi; = 0.5, &lambda; = 1e-4

In [ ]:
pca_value = 8
pi_value = 0.5
lam_value = 1e-4
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 8, &pi; = 0.5, &lambda; = 1e-3

In [ ]:
pca_value = 8
pi_value = 0.5
lam_value = 1e-3
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 8, &pi; = 0.5, &lambda; = 1e-2

In [ ]:
pca_value = 8
pi_value = 0.5
lam_value = 1e-2
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 8, &pi; = 0.5, &lambda; = 1e-1

In [ ]:
pca_value = 8
pi_value = 0.5
lam_value = 1e-1
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 8, &pi; = 0.5, &lambda; = 1.0

In [ ]:
pca_value = 8
pi_value = 0.5
lam_value = 1.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 8, &pi; = 0.5, &lambda; = 10.0

In [ ]:
pca_value = 8
pi_value = 0.5
lam_value = 10.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 8, &pi; = 0.5, &lambda; = 100.0

In [ ]:
pca_value = 8
pi_value = 0.5
lam_value = 100.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 8, &pi; = 0.9, &lambda; = 1e-6

In [ ]:
pca_value = 8
pi_value = 0.9
lam_value = 1e-6
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 8, &pi; = 0.9, &lambda; = 1e-5

In [ ]:
pca_value = 8
pi_value = 0.9
lam_value = 1e-5
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 8, &pi; = 0.9, &lambda; = 1e-4

In [ ]:
pca_value = 8
pi_value = 0.9
lam_value = 1e-4
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 8, &pi; = 0.9, &lambda; = 1e-3

In [ ]:
pca_value = 8
pi_value = 0.9
lam_value = 1e-3
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 8, &pi; = 0.9, &lambda; = 1e-2

In [ ]:
pca_value = 8
pi_value = 0.9
lam_value = 1e-2
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 8, &pi; = 0.9, &lambda; = 1e-1

In [ ]:
pca_value = 8
pi_value = 0.9
lam_value = 1e-1
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 8, &pi; = 0.9, &lambda; = 1.0

In [ ]:
pca_value = 8
pi_value = 0.9
lam_value = 1.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 8, &pi; = 0.9, &lambda; = 10.0

In [ ]:
pca_value = 8
pi_value = 0.9
lam_value = 10.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 8, &pi; = 0.9, &lambda; = 100.0

In [ ]:
pca_value = 8
pi_value = 0.9
lam_value = 100.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 7, &pi; = 0.1, &lambda; = 1e-6

In [ ]:
pca_value = 7
pi_value = 0.1
lam_value = 1e-6
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 7, &pi; = 0.1, &lambda; = 1e-5

In [ ]:
pca_value = 7
pi_value = 0.1
lam_value = 1e-5
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 7, &pi; = 0.1, &lambda; = 1e-4

In [ ]:
pca_value = 7
pi_value = 0.1
lam_value = 1e-4
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 7, &pi; = 0.1, &lambda; = 1e-3

In [ ]:
pca_value = 7
pi_value = 0.1
lam_value = 1e-3
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 7, &pi; = 0.1, &lambda; = 1e-2

In [ ]:
pca_value = 7
pi_value = 0.1
lam_value = 1e-2
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 7, &pi; = 0.1, &lambda; = 1e-1

In [ ]:
pca_value = 7
pi_value = 0.1
lam_value = 1e-1
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 7, &pi; = 0.1, &lambda; = 1.0

In [ ]:
pca_value = 7
pi_value = 0.1
lam_value = 1.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 7, &pi; = 0.1, &lambda; = 10.0

In [ ]:
pca_value = 7
pi_value = 0.1
lam_value = 10.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 7, &pi; = 0.1, &lambda; = 100.0

In [ ]:
pca_value = 7
pi_value = 0.1
lam_value = 100.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 7, &pi; = 0.5, &lambda; = 1e-6

In [ ]:
pca_value = 7
pi_value = 0.5
lam_value = 1e-6
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 7, &pi; = 0.5, &lambda; = 1e-5

In [ ]:
pca_value = 7
pi_value = 0.5
lam_value = 1e-5
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 7, &pi; = 0.5, &lambda; = 1e-4

In [ ]:
pca_value = 7
pi_value = 0.5
lam_value = 1e-4
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 7, &pi; = 0.5, &lambda; = 1e-3

In [ ]:
pca_value = 7
pi_value = 0.5
lam_value = 1e-3
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 7, &pi; = 0.5, &lambda; = 1e-2

In [ ]:
pca_value = 7
pi_value = 0.5
lam_value = 1e-2
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 7, &pi; = 0.5, &lambda; = 1e-1

In [ ]:
pca_value = 7
pi_value = 0.5
lam_value = 1e-1
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 7, &pi; = 0.5, &lambda; = 1.0

In [ ]:
pca_value = 7
pi_value = 0.5
lam_value = 1.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 7, &pi; = 0.5, &lambda; = 10.0

In [ ]:
pca_value = 7
pi_value = 0.5
lam_value = 10.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 7, &pi; = 0.5, &lambda; = 100.0

In [ ]:
pca_value = 7
pi_value = 0.5
lam_value = 100.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 7, &pi; = 0.9, &lambda; = 1e-6

In [ ]:
pca_value = 7
pi_value = 0.9
lam_value = 1e-6
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 7, &pi; = 0.9, &lambda; = 1e-5

In [ ]:
pca_value = 7
pi_value = 0.9
lam_value = 1e-5
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 7, &pi; = 0.9, &lambda; = 1e-4

In [ ]:
pca_value = 7
pi_value = 0.9
lam_value = 1e-4
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 7, &pi; = 0.9, &lambda; = 1e-3

In [ ]:
pca_value = 7
pi_value = 0.9
lam_value = 1e-3
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 7, &pi; = 0.9, &lambda; = 1e-2

In [ ]:
pca_value = 7
pi_value = 0.9
lam_value = 1e-2
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 7, &pi; = 0.9, &lambda; = 1e-1

In [ ]:
pca_value = 7
pi_value = 0.9
lam_value = 1e-1
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 7, &pi; = 0.9, &lambda; = 1.0

In [ ]:
pca_value = 7
pi_value = 0.9
lam_value = 1.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 7, &pi; = 0.9, &lambda; = 10.0

In [ ]:
pca_value = 7
pi_value = 0.9
lam_value = 10.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 7, &pi; = 0.9, &lambda; = 100.0

In [ ]:
pca_value = 7
pi_value = 0.9
lam_value = 100.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 6, &pi; = 0.1, &lambda; = 1e-6

In [ ]:
pca_value = 6
pi_value = 0.1
lam_value = 1e-6
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 6, &pi; = 0.1, &lambda; = 1e-5

In [ ]:
pca_value = 6
pi_value = 0.1
lam_value = 1e-5
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 6, &pi; = 0.1, &lambda; = 1e-4

In [ ]:
pca_value = 6
pi_value = 0.1
lam_value = 1e-4
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 6, &pi; = 0.1, &lambda; = 1e-3

In [ ]:
pca_value = 6
pi_value = 0.1
lam_value = 1e-3
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 6, &pi; = 0.1, &lambda; = 1e-2

In [ ]:
pca_value = 6
pi_value = 0.1
lam_value = 1e-2
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 6, &pi; = 0.1, &lambda; = 1e-1

In [ ]:
pca_value = 6
pi_value = 0.1
lam_value = 1e-1
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 6, &pi; = 0.1, &lambda; = 1.0

In [ ]:
pca_value = 6
pi_value = 0.1
lam_value = 1.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 6, &pi; = 0.1, &lambda; = 10.0

In [ ]:
pca_value = 6
pi_value = 0.1
lam_value = 10.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 6, &pi; = 0.1, &lambda; = 100.0

In [ ]:
pca_value = 6
pi_value = 0.1
lam_value = 100.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 6, &pi; = 0.5, &lambda; = 1e-6

In [ ]:
pca_value = 6
pi_value = 0.5
lam_value = 1e-6
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 6, &pi; = 0.5, &lambda; = 1e-5

In [ ]:
pca_value = 6
pi_value = 0.5
lam_value = 1e-5
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 6, &pi; = 0.5, &lambda; = 1e-4

In [ ]:
pca_value = 6
pi_value = 0.5
lam_value = 1e-4
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 6, &pi; = 0.5, &lambda; = 1e-3

In [ ]:
pca_value = 6
pi_value = 0.5
lam_value = 1e-3
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 6, &pi; = 0.5, &lambda; = 1e-2

In [ ]:
pca_value = 6
pi_value = 0.5
lam_value = 1e-2
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 6, &pi; = 0.5, &lambda; = 1e-1

In [ ]:
pca_value = 6
pi_value = 0.5
lam_value = 1e-1
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 6, &pi; = 0.5, &lambda; = 1.0

In [ ]:
pca_value = 6
pi_value = 0.5
lam_value = 1.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 6, &pi; = 0.5, &lambda; = 10.0

In [ ]:
pca_value = 6
pi_value = 0.5
lam_value = 10.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 6, &pi; = 0.5, &lambda; = 100.0

In [ ]:
pca_value = 6
pi_value = 0.5
lam_value = 100.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 6, &pi; = 0.9, &lambda; = 1e-6

In [ ]:
pca_value = 6
pi_value = 0.9
lam_value = 1e-6
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 6, &pi; = 0.9, &lambda; = 1e-5

In [ ]:
pca_value = 6
pi_value = 0.9
lam_value = 1e-5
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 6, &pi; = 0.9, &lambda; = 1e-4

In [ ]:
pca_value = 6
pi_value = 0.9
lam_value = 1e-4
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 6, &pi; = 0.9, &lambda; = 1e-3

In [ ]:
pca_value = 6
pi_value = 0.9
lam_value = 1e-3
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 6, &pi; = 0.9, &lambda; = 1e-2

In [ ]:
pca_value = 6
pi_value = 0.9
lam_value = 1e-2
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 6, &pi; = 0.9, &lambda; = 1e-1

In [ ]:
pca_value = 6
pi_value = 0.9
lam_value = 1e-1
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 6, &pi; = 0.9, &lambda; = 1.0

In [ ]:
pca_value = 6
pi_value = 0.9
lam_value = 1.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 6, &pi; = 0.9, &lambda; = 10.0

In [ ]:
pca_value = 6
pi_value = 0.9
lam_value = 10.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 6, &pi; = 0.9, &lambda; = 100.0

In [ ]:
pca_value = 6
pi_value = 0.9
lam_value = 100.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 5, &pi; = 0.1, &lambda; = 1e-6

In [ ]:
pca_value = 5
pi_value = 0.1
lam_value = 1e-6
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 5, &pi; = 0.1, &lambda; = 1e-5

In [ ]:
pca_value = 5
pi_value = 0.1
lam_value = 1e-5
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 5, &pi; = 0.1, &lambda; = 1e-4

In [ ]:
pca_value = 5
pi_value = 0.1
lam_value = 1e-4
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 5, &pi; = 0.1, &lambda; = 1e-3

In [ ]:
pca_value = 5
pi_value = 0.1
lam_value = 1e-3
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 5, &pi; = 0.1, &lambda; = 1e-2

In [ ]:
pca_value = 5
pi_value = 0.1
lam_value = 1e-2
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 5, &pi; = 0.1, &lambda; = 1e-1

In [ ]:
pca_value = 5
pi_value = 0.1
lam_value = 1e-1
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 5, &pi; = 0.1, &lambda; = 1.0

In [ ]:
pca_value = 5
pi_value = 0.1
lam_value = 1.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 5, &pi; = 0.1, &lambda; = 10.0

In [ ]:
pca_value = 5
pi_value = 0.1
lam_value = 10.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 5, &pi; = 0.1, &lambda; = 100.0

In [ ]:
pca_value = 5
pi_value = 0.1
lam_value = 100.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 5, &pi; = 0.5, &lambda; = 1e-6

In [ ]:
pca_value = 5
pi_value = 0.5
lam_value = 1e-6
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 5, &pi; = 0.5, &lambda; = 1e-5

In [ ]:
pca_value = 5
pi_value = 0.5
lam_value = 1e-5
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 5, &pi; = 0.5, &lambda; = 1e-4

In [ ]:
pca_value = 5
pi_value = 0.5
lam_value = 1e-4
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 5, &pi; = 0.5, &lambda; = 1e-3

In [ ]:
pca_value = 5
pi_value = 0.5
lam_value = 1e-3
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 5, &pi; = 0.5, &lambda; = 1e-2

In [ ]:
pca_value = 5
pi_value = 0.5
lam_value = 1e-2
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 5, &pi; = 0.5, &lambda; = 1e-1

In [ ]:
pca_value = 5
pi_value = 0.5
lam_value = 1e-1
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 5, &pi; = 0.5, &lambda; = 1.0

In [ ]:
pca_value = 5
pi_value = 0.5
lam_value = 1.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 5, &pi; = 0.5, &lambda; = 10.0

In [ ]:
pca_value = 5
pi_value = 0.5
lam_value = 10.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 5, &pi; = 0.5, &lambda; = 100.0

In [ ]:
pca_value = 5
pi_value = 0.5
lam_value = 100.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 5, &pi; = 0.9, &lambda; = 1e-6

In [ ]:
pca_value = 5
pi_value = 0.9
lam_value = 1e-6
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 5, &pi; = 0.9, &lambda; = 1e-5

In [ ]:
pca_value = 5
pi_value = 0.9
lam_value = 1e-5
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 5, &pi; = 0.9, &lambda; = 1e-4

In [ ]:
pca_value = 5
pi_value = 0.9
lam_value = 1e-4
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 5, &pi; = 0.9, &lambda; = 1e-3

In [ ]:
pca_value = 5
pi_value = 0.9
lam_value = 1e-3
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 5, &pi; = 0.9, &lambda; = 1e-2

In [ ]:
pca_value = 5
pi_value = 0.9
lam_value = 1e-2
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 5, &pi; = 0.9, &lambda; = 1e-1

In [ ]:
pca_value = 5
pi_value = 0.9
lam_value = 1e-1
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 5, &pi; = 0.9, &lambda; = 1.0

In [ ]:
pca_value = 5
pi_value = 0.9
lam_value = 1.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 5, &pi; = 0.9, &lambda; = 10.0

In [ ]:
pca_value = 5
pi_value = 0.9
lam_value = 10.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")

PCA = 5, &pi; = 0.9, &lambda; = 100.0

In [ ]:
pca_value = 5
pi_value = 0.9
lam_value = 100.0
dis_kfold(DTR, LTR, pca_value, pi_value, lam_value, quadratic_logistic_regression, "Quadratic Logistic Regression")